# AdventureWorks Relational Postgres Lab

### Introduction

In this lesson, we will work with the adventureworks database in postgres.  Let's get started.

### Loading our data


We can begin by making sure our postgres application is running (if on a Mac, look for the elephant at the top right of the screen).  Once running we'll walk through the instructions in the [Adventureworks for postgres](https://github.com/lorint/AdventureWorks-for-Postgres) repo below.

First move into the `install-script` directory.  Then run the following:

```bash
psql -c "CREATE DATABASE \"adventureworks\";"
psql -d adventureworks < install.sql
```

Next connect to the adventureworks database.

```sql
\c "Adventureworks"
```

One confusing thing is that if we simply type `\dt` to display our tables, we won't find any listed.  This is because all of our tables are under different schemas.  We can see this if, while connected to the adventureworks database, we run the following:

```sql
\dt (humanresources|person|production|purchasing|sales).*
```

So, as we can see, there are indeed a lot of tables in our database.  And we can query any of those tables with the schema prefix like so.

```sql
select * from person.address limit 1;
```

The database is pretty complicated, and you can see all of the relations from the [erd link](https://i0.wp.com/improveandrepeat.com/wp-content/uploads/2018/12/AdvWorksOLTPSchemaVisio.png?ssl=1).  But we will stick to the tables in the `sales` schema.  Let's get started.



### Beginning our queries

We can connect to our database using the postgres library.

In [1]:
# !pip install pandas psycopg2



In [3]:
import pandas as pd
import psycopg2

conn = psycopg2.connect(database="adventureworks", user="postgres")


And from there, we can use pandas to read from our database.  We'll can begin with our customers table.

In [5]:
pd.read_sql("select * from sales.Customer limit 3", conn)

/var/folders/qc/gjj9ts396vnf0zzl4fk7vprw0000gn/T/ipykernel_35076/1040193723.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql("select * from sales.Customer limit 3", conn)


,customerid,personid,storeid,territoryid,rowguid,modifieddate
0,1,None,934,1,3f5ae95e-b87d-4aed-95b4-c3797afcb74f,2014-09-12 11:15:07.263
1,2,None,1028,1,e552f657-a9af-4a7d-a645-c429d6e02491,2014-09-12 11:15:07.263
2,3,None,642,4,130774b1-db21-4ef3-98c8-c104bcd6ed6d,2014-09-12 11:15:07.263


We can hide warnings by running the following code.

In [6]:
import warnings
warnings.filterwarnings('ignore')

And then let's view some of the data in our `SalesOrderHeader` table.

> Notice below we have a `.T` at the very end.  This is to `transpose` our dataframe -- which sometimes makes it easier to see all of the columns.  The `0` and `1` are to describe each of the rows of data.

In [8]:
pd.read_sql("select * from sales.SalesOrderHeader limit 2", conn).T

,0,1
salesorderid,43659,43660
revisionnumber,8,8
orderdate,2011-05-31 00:00:00,2011-05-31 00:00:00
duedate,2011-06-12 00:00:00,2011-06-12 00:00:00
shipdate,2011-06-07 00:00:00,2011-06-07 00:00:00
status,5,5
onlineorderflag,False,False
purchaseordernumber,PO522145787,PO18850127500
accountnumber,10-4020-000676,10-4020-000117
customerid,29825,29672


So we can see that our `SalesOrderHeader` table has a good amount of foreign keys including `customerId`.

The SalesOrderHeader table is one key table.  Another is the `salesOrderDetail` table.  Make sure to take a look at the [erd link](https://i0.wp.com/improveandrepeat.com/wp-content/uploads/2018/12/AdvWorksOLTPSchemaVisio.png?ssl=1) to get a sense of how the tables and others in the sales schema relate to one another.

### Writing some queries

Noe let's get a sense of our data.  First, find the earliest date of our orders in the `SalesOrderHeader` table.

In [10]:
import pandas as pd
query = "select min(orderdate) from sales.SalesOrderHeader;"
pd.read_sql(query, conn)

# 	min
# 0	2011-05-31

,min
0,2011-05-31


And next find the latest date of the orders.

In [11]:
query = "select max(orderdate) from sales.SalesOrderHeader;"
pd.read_sql(query, conn)


# max
# 0	2014-06-30

,max
0,2014-06-30


We can see that our most recent orders were placed at the end June of 2014.  

Next find the the top five total amounts spent by customers since July 1 2013.  Return both the customerIds and the total amounts spent.

In [15]:
query = """select customerid, sum(totaldue) as total_amount
from sales.SalesOrderHeader
where orderdate > '2013-06-30'
group by 1
order by 2 desc
limit 5;"""
pd.read_sql(query, conn)

# 	customerid	total_amount
# 0	29641	419331.7837
# 1	29629	389437.6577
# 2	29923	386059.4979
# 3	29736	363872.0389
# 4	29701	355051.0477

,customerid,total_amount
0,29641,419331.7837
1,29629,389437.6577
2,29923,386059.4979
3,29736,363872.0389
4,29701,355051.0477


Then find the names of the top five products that brought in the most amount of revenue, and the amount of revenue for each since `July 1st 2013` -- we can use the `salesOrderDetail.modifiedDate` column as a proxy for the dates of each sale. 

In [18]:
pd.read_sql("select * from sales.SalesOrderDetail limit 2", conn).T

,0,1
salesorderid,43659,43659
salesorderdetailid,1,2
carriertrackingnumber,4911-403C-98,4911-403C-98
orderqty,1,3
productid,776,777
specialofferid,1,1
unitprice,2024.994,2024.994
unitpricediscount,0.0,0.0
rowguid,b207c96d-d9e6-402b-8470-2cc176c42283,7abb600d-1e77-41be-9fe5-b9142cfc08fa
modifieddate,2011-05-31 00:00:00,2011-05-31 00:00:00


In [21]:
query = """select p.name, sum(s.unitprice) 
from sales.SalesOrderDetail as s
join production.Product as p on s.productid = p.productid
where s.modifieddate > '2013-06-30'
group by 1
order by 2 desc
limit 5;
"""
pd.read_sql(query, conn)
# product.productid
#product.name
#salesorderdetail.product id to product table
#salesorderdetail.salesorderid = sales order table for price

# name	total_price
# 0	Mountain-200 Black, 38	1.259927e+06
# 1	Mountain-200 Black, 42	1.222931e+06
# 2	Mountain-200 Silver, 38	1.156979e+06
# 3	Mountain-200 Black, 46	1.141849e+06
# 4	Mountain-200 Silver, 46	1.128350e+06

,name,sum
0,"Mountain-200 Black, 38",1.262222e+06
1,"Mountain-200 Black, 42",1.222931e+06
2,"Mountain-200 Silver, 38",1.156979e+06
3,"Mountain-200 Black, 46",1.141849e+06
4,"Mountain-200 Silver, 46",1.128350e+06


So now that we've done some analysis on the products, let's take a look at the regions.  Find the territoryids related to the territories that have had the most sales in the past year.

In [29]:
pd.read_sql('select * from sales.SalesTerritory limit 5;', conn).T

,0,1,2,3,4
territoryid,1,2,3,4,5
name,Northwest,Northeast,Central,Southwest,Southeast
countryregioncode,US,US,US,US,US
group,North America,North America,North America,North America,North America
salesytd,7887186.7882,2402176.8476,3072175.118,10510853.8739,2538667.2515
saleslastyear,3298694.4938,3607148.9371,3205014.0767,5366575.7098,3925071.4318
costytd,0.0,0.0,0.0,0.0,0.0
costlastyear,0.0,0.0,0.0,0.0,0.0
rowguid,43689a10-e30b-497f-b0de-11de20267ff7,00fb7309-96cc-49e2-8363-0a1ba72486f2,df6e7fd8-1a8d-468c-b103-ed8addb452c1,dc3e9ea0-7950-4431-9428-99dbcbc33865,6dc4165a-5e4c-42d2-809d-4344e0ac75e7
modifieddate,2008-04-30 00:00:00,2008-04-30 00:00:00,2008-04-30 00:00:00,2008-04-30 00:00:00,2008-04-30 00:00:00


In [32]:
query = """
select t.name, sum(soh.totaldue) as total_amount
from sales.SalesTerritory as t
join sales.SalesOrderHeader as soh on t.territoryid = soh.territoryid
where soh.orderdate > '2013-06-30'
group by 1
order by 2 desc
"""
pd.read_sql(query, conn)

# name	total_amount
# 0	Southwest	9.994438e+06
# 1	Northwest	7.492804e+06
# 2	Canada	6.367257e+06
# 3	Australia	6.154403e+06
# 4	United Kingdom	5.047151e+06
# 5	France	4.370710e+06
# 6	Germany	3.630621e+06
# 7	Central	2.791293e+06
# 8	Southeast	2.351479e+06
# 9	Northeast	2.168895e+06

,name,total_amount
0,Southwest,9.994744e+06
1,Northwest,7.498216e+06
2,Canada,6.368789e+06
3,Australia,6.159413e+06
4,United Kingdom,5.048687e+06
5,France,4.376123e+06
6,Germany,3.630800e+06
7,Central,2.791293e+06
8,Southeast,2.351479e+06
9,Northeast,2.168895e+06


### Looking at Salespeople

Next, let's try to find the top performing salespeople.  We can begin by looking at our salesperson table.

In [33]:
pd.read_sql("""select * from sales.SalesPerson limit 2""", conn).T

,0,1
businessentityid,274,275
territoryid,NaN,2.0
salesquota,NaN,300000.0
bonus,0.0,4100.0
commissionpct,0.0,0.012
salesytd,559697.5639,3763178.1787
saleslastyear,0.0,1750406.4785
rowguid,48754992-9ee0-4c0e-8c94-9451604e3e02,1e0a7274-3064-4f58-88ee-4c6586c87169
modifieddate,2010-12-28 00:00:00,2011-05-24 00:00:00


We can see that we have a `salesytd` column as well as a `modifieddate` column.  One item that may be difficult to identify is the sales person id.  This it turns out is the `businessentityid`.  

Now let's find the salesperson id salesytd of the salesperson who had the most sales in the `Southwest` territory, our best performing territory.

In [42]:
query = """
select s.businessentityid, s.salesytd
from sales.SalesPerson as s
join sales.SalesTerritory as t on s.territoryid = t.territoryid
where t.name = 'Southwest'
order by 2 desc
limit 1
"""

pd.read_sql(query, conn)

#  businessentityid	salesytd
# 0	276	4.251369e+06

,businessentityid,salesytd
0,276,4.251369e+06


Then let's the top three items that the salesperson with id 276 sold.  

> Some of the foreign keys may be tricky to figure out but use your best guess.

In [46]:
query = """
select sod.orderqty, p.name
from sales.SalesPerson as sp
join sales.SalesOrderHeader as soh on soh.salespersonid = sp.businessentityid
join sales.SalesOrderDetail as sod on sod.salesorderid = soh.salesorderid
join production.Product as p on p.productid = sod.productid
where sp.businessentityid = 276
limit 3
"""
pd.read_sql(query, conn)

# 	amount	name
# 0	141	Long-Sleeve Logo Jersey, L
# 1	135	AWC Logo Cap
# 2	124	Sport-100 Helmet, Blue

,unitprice,name
0,419.4589,"Road-650 Red, 60"
1,419.4589,"Road-650 Red, 52"
2,2146.9620,"Road-150 Red, 56"


### Summary 

In this lesson, we practiced working with our relational queries.  The adventureworks database is excellent for practicing postgres skills so feel free to continue practicing and exploring the various schemas.